In [2]:
from jinja2 import DictLoader, Environment, FileSystemLoader
#from lxml.etree import DTD
import os

In [3]:
# Parametros para funciones generadoras
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)

def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

In [4]:
# Defino funciones generadoras de codigo
# 
def generateF(template, fType, stock, variables, formula):
    # Nota: explicacion de parametros que toma generateF ==>
    # template : nombre del template a utilizar (debe haber un .cpp y un .h con este nombre)
    # fType : 'fPlus' o 'fMinus' son los dos tipos de funciones que se pueden declarar
    # stock : nombre del 'stock' asociado a esta funcion (el nombre de la variable que esta siendo integrada)
    # variables : nombres de las variables de estado utilizadas en la funcion
    # formula : formula de la outputFunction, que utiliza las variables de estado del atomico
    
    # Template names
    templateH   = template + '.h'
    templateCPP = template + '.cpp'
    
    # Derived parameters for template
    stockName = fType + stock[0].upper() + stock[1:]
    stockNameConstant = stockName.upper()
    setVariables  = list(map(lambda var : 'isSet' + var[0].upper() + var[1:], variables))
    inPorts       = list(map(lambda var : 'in' + var[0].upper() + var[1:], variables))
    context = {
        'stockName' : stockName,
        'stockNameConstant' : stockNameConstant,
        'variables' : variables,
        'setVariables' : setVariables,
        'inPorts' : inPorts,
        'formula' : formula
    }
    hFile = render_template(templateH, context)
    cppFile = render_template(templateCPP, context)
    return {"h" : hFile, "cpp" : cppFile}

def generateReg(template, stockNames):
    stockNamesConstant = list(map(lambda var : var.upper(), stockNames))
    templateCPP = template + '.cpp'
    context = { 'stockNames' : stockNames, 'stockNamesConstant' : stockNamesConstant }
    cppFile = render_template(templateCPP, context)
    return cppFile

def generateFiles(folder, atomics):
    for atomic in atomics:
        f = generateF(atomic[0], atomic[1], atomic[2], atomic[3], atomic[4])
        hFile   = f['h']
        cppFile = f['cpp']
        # Genero hFile y cppFile
        with open(folder + '/' + atomic[1] + atomic[2][0].upper() + atomic[2][1:] + '.h', 'w') as f:
            f.write(hFile)
        with open(folder + '/' + atomic[1] + atomic[2][0].upper() + atomic[2][1:] + '.cpp', 'w') as f:
            f.write(cppFile)
    # Genero 'reg.cpp'
    atomicNames = [atomic[1] + atomic[2][0].upper() + atomic[2][1:] for atomic in atomics]
    regFile = generateReg('reg', atomicNames)
    with open(folder + '/reg.cpp', 'w') as f:
        f.write(regFile)

In [5]:
# Nota : 'Fpm' se refiera a (Fplus + Fminus)
# TODO : matchear el llamado a estas funciones con los archivos .devsml y coordinar con el .ma
# Teacup
atomicsTeacup = [
    ['Fpm', 'Fminus', 'teacupTemperature', ['teacupTemperature', 'characteristicTime', 'roomTemperature'], 
     '(teacupTemperature - roomTemperature) / characteristicTime']
]
generateFiles('teacup', atomicsTeacup)

# Lotka-volterra
atomicsLV = [
    ['Fpm', 'Fminus', 'lynx', ['lynx', 'lynxDeathRate'], 'lynxDeathRate * lynx'],
    ['Fpm', 'Fplus',  'lynx', ['lynx', 'hares', 'lynxBirthRate'], 'lynxBirthRate * lynx * hares'],
    ['Fpm', 'Fminus', 'hares', ['hares', 'lynx', 'hareDeathRate'], 'hareDeathRate * hares * lynx'],
    ['Fpm', 'Fplus',  'hares', ['hares', 'hareBirthRate'], 'hareBirthRate * hares']
]
generateFiles('lotka-volterra', atomicsLV)

# SIR
atomicsSIR = [
    ['Fpm', 'Fminus', 'infected', ['infected', 'duration'], 'infected / duration'],
    ['Fpm', 'Fplus',  'infected', ['infected', 'susceptible', 'contactInfectivity', 'totalPopulation'],
     '(infected * susceptible) / (contactInfectivity * totalPopulation)'],
    ['Fpm', 'Fminus', 'susceptible', ['susceptible', 'infected', 'contactInfectivity', 'totalPopulation'],
     '(infected * susceptible) / (contactInfectivity * totalPopulation)'],
    ['Fpm', 'Fplus',  'recovered', ['infected', 'duration'], 'infected / duration']
]
generateFiles('SIR', atomicsSIR)

In [188]:
# Examples:

# 0) Register atomic models
#print(generateReg('reg', ['FplusLynx', 'FminusLynx']))

# 1) Teacup
#print(generateF('Fpm', 'Fminus', 'teacup', ['teacupTemperature', 'characteristicTime', 'roomTemperature'], 
#                '(teacupTemperature - roomTemperature) / characteristicTime')['h'])
#print(generateF('Fpm', 'Fminus', 'teacup', ['teacupTemperature', 'characteristicTime', 'roomTemperature'], 
#                '(teacupTemperature - roomTemperature) / characteristicTime')['cpp'])

# 2) Lotka-Volterra
# Lynx
#print(generateF('Fpm', 'Fminus', 'lynx', ['lynx', 'lynxDeathRate'], 'lynxDeathRate * lynx')['h'])
#print(generateF('Fpm', 'Fminus', 'lynx', ['lynx', 'lynxDeathRate'], 'lynxDeathRate * lynx')['cpp'])
#print(generateF('Fpm', 'Fplus', 'lynx', ['lynx', 'hares', 'lynxBirthRate'], 'lynxBirthRate * lynx * hares')['h'])
#print(generateF('Fpm', 'Fplus', 'lynx', ['lynx', 'hares', 'lynxBirthRate'], 'lynxBirthRate * lynx * hares')['cpp'])

# Hares
#print(generateF('Fpm', 'Fminus', 'hares', ['hares', 'lynx', 'hareDeathRate'], 'hareDeathRate * hares * lynx')['h'])
#print(generateF('Fpm', 'Fminus', 'hares', ['hares', 'lynx', 'hareDeathRate'], 'hareDeathRate * hares * lynx')['cpp'])
#print(generateF('Fpm', 'Fplus', 'hares', ['hares', 'hareBirthRate'], 'hareBirthRate * hares')['h'])
#print(generateF('Fpm', 'Fplus', 'hares', ['hares', 'hareBirthRate'], 'hareBirthRate * hares')['cpp'])

# 3) SIR
# Infected
#print(generateF('Fpm', 'Fminus', 'infected', ['infected', 'duration'], 'infected / duration')['h'])
#print(generateF('Fpm', 'Fminus', 'infected', ['infected', 'duration'], 'infected / duration')['cpp'])
#print(generateF('Fpm', 'Fplus', 'infected', ['infected', 'susceptible', 'contactInfectivity', 'totalPopulation'],
#                '(infected * susceptible) / (contactInfectivity * totalPopulation)')['h'])
#print(generateF('Fpm', 'Fplus', 'infected', ['infected', 'susceptible', 'contactInfectivity', 'totalPopulation'],
#                '(infected * susceptible) / (contactInfectivity * totalPopulation)')['cpp'])

# Susceptible
#print(generateF('Fpm', 'Fminus', 'susceptible', ['susceptible', 'infected', 'contactInfectivity', 'totalPopulation'],
#                '(infected * susceptible) / (contactInfectivity * totalPopulation)')['h'])
#print(generateF('Fpm', 'Fminus', 'susceptible', ['susceptible', 'infected', 'contactInfectivity', 'totalPopulation'],
#                '(infected * susceptible) / (contactInfectivity * totalPopulation)')['cpp'])

# Recovered
#print(generateF('Fpm', 'Fplus', 'recovered', ['infected', 'duration'], 'infected / duration')['h'])
#print(generateF('Fpm', 'Fplus', 'recovered', ['infected', 'duration'], 'infected / duration')['cpp'])